<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
 <h2>Mentoría: Clasificación diagnóstica de mamografías </h2>
</center>

<h3> Práctico II -  Análisis y Curación <h3>
<h4>Integrantes: Mario Agustín Sgró, Lucía Benítez, Germán Bertea y Carolina Díaz <h4>

**Objetivo y alcance:** 
    Aprendizaje sobre limpieza y mejora de la base de datos. 
    Manejo y entendimiento de algunas técnicas de aumento de información, manejo de datos faltantes, imputación y reducción de dimensionalidad. 
Método: Se utilizarán componentes principales, algoritmo de k-medias, distancia entre observaciones, k-vecinos más cercanos, escalado de variables/features.

**Estructura del informe:**
Presentar en un archivo jupyter notebook con la resolución detallada de las siguientes consignas:

1- Analizar los tipos de datos y si hay algun inconveniente con la codificación de las variables.

2- Identificar los datos faltantes de la base de datos.

3- Realizar un listado con resumen explicativo de algunas técnicas de tratamiento de datos faltantes. 

4- Elegir tres abordajes (con alguna variante de metapámetros si las tuviera) incluída la imputación usando el vecino más cercano. 

5- Investigar opciones de aumentación de información, como por ejemplo polinómica de algunas variables, implementar algunas y evaluar su aporte en la clasificación.

6- Realizar un reescalado de las variables utilizando min-max y Z-score.

7- Realizar reducción de dimensionalidad usando componentes principales y otra que considere adecuada.

8- Combinar los abordajes.


In [46]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sns
import chardet
import string


plt.rcParams['figure.figsize'] = (200, 200)

In [10]:
if sns.__version__ != '0.9.0':
    print('Atención! utilizamos seaborn versión 0.9.0')

In [11]:
pd.options.display.max_columns = 200
#pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 1000

ANÁLISIS EXPLORATORIO

Comenzamos la lectura del dataset:

In [151]:
#dataframe = pd.read_csv('datos/Datos_Mamografias.csv',sep=',',index_col=['Imagen'])
dataframe = pd.read_csv('datos/Datos_Mamografias.csv',sep=',')
dataframe.head(3)

,Imagen,Age,View,Laterality,Dcm_1,Dcm_2,Dcm_3,Dcm_4,Dcm_5,Dcm_6,Dcm_7,Dcm_8,Dcm_9,Dcm_10,Dcm_11,Dcm_12,Dcm_13,Dcm_14,Dcm_15,Dcm_16,Dcm_17,Dcm_18,Dcm_19,Dcm_20,Dcm_21,Dcm_22,Dcm_23,Area,Promedio,Media,Desvio,Asimetria,Kurtosis,Moda,Desvio_L1,Desvio_L2,Desvio_L3,Asimetria_L1,Asimetria_L2,Asimetria_L3,promGradiente,Relacion,Relacion_L1,Relacion_L2,Relacion_L3,Relacion_L4,EP,EPBinaria,RelEntropia,Haralick_1,Haralick_2,Haralick_3,Haralick_4,Haralick_5,Haralick_6,Haralick_7,Haralick_8,Haralick_9,Haralick_10,Haralick_11,Haralick_12,Haralick_13,Haralick_14,Haralick_15,Haralick_16,Haralick_17,Haralick_18,Haralick_19,Haralick_20,Haralick_21,Haralick_22,Haralick_23,Haralick_24,Haralick_25,Haralick_26,Haralick_27,Haralick_28,Haralick_29,Haralick_30,Haralick_31,Haralick_32,Haralick_33,Haralick_34,Haralick_35,Haralick_36,Haralick_37,Haralick_38,Haralick_39,Haralick_40,Haralick_41,Haralick_42,Haralick_43,Haralick_44,Haralick_45,Haralick_46,Haralick_47,Haralick_48,Haralick_49,Haralick_50,Haralick_51,Haralick_52,DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14,DF15,DF16,DF17,DF18,DF19,DF20,DF21,DF22,DF23,DF24,DF25,DF26,DF27,DF28,DF29,DF30,DF31,DF32,DF33,DF34,DF35,DF36,DF37,DF38,DF39,DF40,DF41,DF42,DF43,DF44,DF45,DF46,DF47,DF48,DFb1,DFb2,DFb3,DFb4,DFb5,DFb6,DFb7,DFb8,DFb9,DFb10,DFb11,DFb12,DFb13,DFb14,DFb15,DFb16,DFb17,DFb18,DFb19,DFb20,DFb21,DFb22,DFb23,DFb24,DFb25,DFb26,DFb27,DFb28,DFb29,DFb30,DFb31,DFb32,DFb33,DFb34,DFb35,DFb36,DFb37,DFb38,DFb39,DFb40,DFb41,DFb42,DFb43,DFb44,DFb45,DFb46,DFb47,DFb48,ACR
0,Imagen_0.npy,47.0,CC,R,29.0,130.0,155.0,1115.0,RHODIUM,0.05,0.3,TUNGSTEN,54.0,58.5,700.0,621.0,1.073,-8.4,AUTOMATIC,AutoFilter,321.0,6.53,0.0172,0.531,2047.0,4096.0,NO,1906717.0,1204.431581,1108.0,651.401605,0.631631,-0.158630,771.0,582.461892,617.414299,571.626601,1.254898,0.340503,0.478727,26.263890,50.872416,30.246896,39.025868,57.284054,56.462588,0.769471,0.176043,0.012500,0.000003,29632.094980,0.965015,423492.945587,0.010546,2407.880580,1.664340e+06,12.237075,18.933134,8.625112e-07,8.446691,-0.209392,0.994013,0.000002,35926.620735,0.957571,423372.418259,0.009612,2407.639224,1.657563e+06,12.235604,19.052746,7.760232e-07,8.586088,-0.198048,0.992383,0.000003,30038.183717,0.964600,424267.419997,0.010677,2408.826967,1.667031e+06,12.238215,18.941108,8.581286e-07,8.455624,-0.208819,0.993943,0.000002,35106.380011,0.958552,423500.102813,0.009655,2407.789176,1.658894e+06,12.236029,19.042339,7.841840e-07,8.570271,-0.199065,0.992547,1.304171,0.026670,226.278823,41.339488,-0.004142,-0.801469,1.739567,0.322655,571.489193,158.341050,-1.165407,-0.038599,1.770398,0.299554,1114.887169,161.670179,-1.183844,0.085194,1.720933,0.239346,1706.529132,189.689059,-1.103810,0.225300,1.563104,0.110528,2449.700145,306.298340,1.157895,1.152985,1.748927,0.349874,1941.302019,416.666606,0.676891,0.995952,1.852387,0.649428,1560.111365,524.693103,0.226560,0.823606,1.911039,0.972917,1231.635371,639.340074,-0.129694,0.658866,1.347935,0.041248,303.067490,57.467044,2.316524,0.466817,1.704150,0.247506,768.162586,262.713541,-0.122940,-0.589775,1.798841,0.421929,1148.854002,337.062943,-0.678163,0.150996,1.751035,0.341997,1627.667484,380.140389,-0.268313,0.277035,1.569299,0.124358,2030.372695,254.066279,8.731229,-1.076601,1.679952,0.224516,1405.837251,235.788707,5.110653,-0.352031,1.663799,0.207258,867.106498,193.95269,14.221943,0.852135,1.362630,0.046754,334.152548,266.144497,42.727260,5.647393,c
1,Imagen_1.npy,47.0,MLO,R,28.0,100.0,115.0,1102.0,RHODIUM,0.05,0.3,TUNGSTEN,47.0,86.0,700.0,628.0,1.073,-44.9,AUTOMATIC,AutoFilter,301.0,4.33,0.0126,0.519,2047.0,4096.0,NO,1993379.0,1095.674679,966.0,617.925388,0.822188,0.159056,771.0,577.999696,630.151155,590.633212,1.092861,0.499981,0.421337,26.204560,50.051997,35.612135,48.703866,58.672811,50.210659,0.771289,0.165031,0.053571,0.000003,40089.941086,0.947479,381659.322432,0.008924,2191.233824,1.486547e+06,12.113787,18.843584,7.420714e-07,8.663215,-0.180066,0.987916,0.000003,48232.540197,0.936822,381719.250143,0.008153,2191.

In [64]:
print("El data set consiste de ",dataframe.shape[0]," imagenes, con ",dataframe.shape[1]," registros cada una.")

El data set consiste de  6613  imagenes, con  198  registros cada una.


Nombres de las variables/features/características y tipo de variable para cada una:

In [39]:
#for columna, tipo in zip(dataframe.columns.values,dataframe.dtypes):
    #print("%15s %9s" % (columna,tipo))

Debido a que en las columnas 'Dcm_18' y 'Dcm_19' se observaron datos faltantes denotados como 'None',  se ha cambiado por 'np.nan' para que se puedan leer como números los datos de tales columnas:

In [152]:
dataframe['Dcm_18'].replace('None',np.nan,inplace=True)
dataframe['Dcm_19'].replace('None',np.nan,inplace=True)

dataframe.Dcm_18 = dataframe.Dcm_18.astype(float)
dataframe.Dcm_19 = dataframe.Dcm_19.astype(float)

In [153]:
var_float = dataframe.select_dtypes(include=[np.float])
var_no_float = dataframe.select_dtypes(exclude=[np.number])
var_float.shape,var_no_float.shape

print('Número de filas y columnas' + str('\n')+
      '   variables numéricas     '+ str(var_float.shape) + str('\n')
      + '   variables discretas  ' + str(var_no_float.shape))

Número de filas y columnas
   variables numéricas     (6613, 189)
   variables discretas  (6613, 9)


La base de datos contiene 9 variables categóricas y 189 variables numéricas, con 6394 filas de datos

In [154]:
_ntot = dataframe.size
_nmissing = dataframe.isna().sum().sum()
print("Porcentaje de datos faltantes: {:.03%}".format(_nmissing/float(_ntot)))

Porcentaje de datos faltantes: 0.186%


Verificamos que no hay datos duplicados:

In [31]:
dataframe[dataframe.duplicated()]

,Imagen,Age,View,Laterality,Dcm_1,Dcm_2,Dcm_3,Dcm_4,Dcm_5,Dcm_6,Dcm_7,Dcm_8,Dcm_9,Dcm_10,Dcm_11,Dcm_12,Dcm_13,Dcm_14,Dcm_15,Dcm_16,Dcm_17,Dcm_18,Dcm_19,Dcm_20,Dcm_21,Dcm_22,Dcm_23,Area,Promedio,Media,Desvio,Asimetria,Kurtosis,Moda,Desvio_L1,Desvio_L2,Desvio_L3,Asimetria_L1,Asimetria_L2,Asimetria_L3,promGradiente,Relacion,Relacion_L1,Relacion_L2,Relacion_L3,Relacion_L4,EP,EPBinaria,RelEntropia,Haralick_1,Haralick_2,Haralick_3,Haralick_4,Haralick_5,Haralick_6,Haralick_7,Haralick_8,Haralick_9,Haralick_10,Haralick_11,Haralick_12,Haralick_13,Haralick_14,Haralick_15,Haralick_16,Haralick_17,Haralick_18,Haralick_19,Haralick_20,Haralick_21,Haralick_22,Haralick_23,Haralick_24,Haralick_25,Haralick_26,Haralick_27,Haralick_28,Haralick_29,Haralick_30,Haralick_31,Haralick_32,Haralick_33,Haralick_34,Haralick_35,Haralick_36,Haralick_37,Haralick_38,Haralick_39,Haralick_40,Haralick_41,Haralick_42,Haralick_43,Haralick_44,Haralick_45,Haralick_46,Haralick_47,Haralick_48,Haralick_49,Haralick_50,Haralick_51,Haralick_52,DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14,DF15,DF16,DF17,DF18,DF19,DF20,DF21,DF22,DF23,DF24,DF25,DF26,DF27,DF28,DF29,DF30,DF31,DF32,DF33,DF34,DF35,DF36,DF37,DF38,DF39,DF40,DF41,DF42,DF43,DF44,DF45,DF46,DF47,DF48,DFb1,DFb2,DFb3,DFb4,DFb5,DFb6,DFb7,DFb8,DFb9,DFb10,DFb11,DFb12,DFb13,DFb14,DFb15,DFb16,DFb17,DFb18,DFb19,DFb20,DFb21,DFb22,DFb23,DFb24,DFb25,DFb26,DFb27,DFb28,DFb29,DFb30,DFb31,DFb32,DFb33,DFb34,DFb35,DFb36,DFb37,DFb38,DFb39,DFb40,DFb41,DFb42,DFb43,DFb44,DFb45,DFb46,DFb47,DFb48,ACR


Como Pandas acepta valores duplicados en los índices también debemos verificar ahí:

In [34]:
pd.Series(dataframe.index,dtype=str).describe()

count     6613
unique    6613
top       5089
freq         1
dtype: object

In [35]:
dataframe[dataframe.index.duplicated()]

,Imagen,Age,View,Laterality,Dcm_1,Dcm_2,Dcm_3,Dcm_4,Dcm_5,Dcm_6,Dcm_7,Dcm_8,Dcm_9,Dcm_10,Dcm_11,Dcm_12,Dcm_13,Dcm_14,Dcm_15,Dcm_16,Dcm_17,Dcm_18,Dcm_19,Dcm_20,Dcm_21,Dcm_22,Dcm_23,Area,Promedio,Media,Desvio,Asimetria,Kurtosis,Moda,Desvio_L1,Desvio_L2,Desvio_L3,Asimetria_L1,Asimetria_L2,Asimetria_L3,promGradiente,Relacion,Relacion_L1,Relacion_L2,Relacion_L3,Relacion_L4,EP,EPBinaria,RelEntropia,Haralick_1,Haralick_2,Haralick_3,Haralick_4,Haralick_5,Haralick_6,Haralick_7,Haralick_8,Haralick_9,Haralick_10,Haralick_11,Haralick_12,Haralick_13,Haralick_14,Haralick_15,Haralick_16,Haralick_17,Haralick_18,Haralick_19,Haralick_20,Haralick_21,Haralick_22,Haralick_23,Haralick_24,Haralick_25,Haralick_26,Haralick_27,Haralick_28,Haralick_29,Haralick_30,Haralick_31,Haralick_32,Haralick_33,Haralick_34,Haralick_35,Haralick_36,Haralick_37,Haralick_38,Haralick_39,Haralick_40,Haralick_41,Haralick_42,Haralick_43,Haralick_44,Haralick_45,Haralick_46,Haralick_47,Haralick_48,Haralick_49,Haralick_50,Haralick_51,Haralick_52,DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14,DF15,DF16,DF17,DF18,DF19,DF20,DF21,DF22,DF23,DF24,DF25,DF26,DF27,DF28,DF29,DF30,DF31,DF32,DF33,DF34,DF35,DF36,DF37,DF38,DF39,DF40,DF41,DF42,DF43,DF44,DF45,DF46,DF47,DF48,DFb1,DFb2,DFb3,DFb4,DFb5,DFb6,DFb7,DFb8,DFb9,DFb10,DFb11,DFb12,DFb13,DFb14,DFb15,DFb16,DFb17,DFb18,DFb19,DFb20,DFb21,DFb22,DFb23,DFb24,DFb25,DFb26,DFb27,DFb28,DFb29,DFb30,DFb31,DFb32,DFb33,DFb34,DFb35,DFb36,DFb37,DFb38,DFb39,DFb40,DFb41,DFb42,DFb43,DFb44,DFb45,DFb46,DFb47,DFb48,ACR


A su vez, chequeamos que no haya caracteres fuera de a-Z, 0-9 y _ en los nombres de columnas

In [48]:
dataframe.columns[~dataframe.columns.str.match(r'^(\w+)$')]

Index([], dtype='object')

In [9]:
#Dentro de la base de datos hay filas que contienen todos sus campos sin registro, y es por ello que las descartamos de los datos.
#data = dataframe.dropna(axis=0)  
#contar cuántos nan y restar del total  ver si son los datos que faltan

Distinguimos las columnas y datos faltantes:

In [67]:
null_columns=dataframe.columns[dataframe.isnull().any()]
dataframe[null_columns].isnull().sum()

Age                8
View               8
Laterality         8
Dcm_1              8
Dcm_2              8
Dcm_3              8
Dcm_4              8
Dcm_5              8
Dcm_6              8
Dcm_7              8
Dcm_8              8
Dcm_9              8
Dcm_10             8
Dcm_11             8
Dcm_12             8
Dcm_13             8
Dcm_14             8
Dcm_15             8
Dcm_16             8
Dcm_17             8
Dcm_18            22
Dcm_19            22
Dcm_20             8
Dcm_21             8
Dcm_22             8
Dcm_23             8
Area               8
Promedio           8
Media              8
Desvio             8
Asimetria          8
Kurtosis           8
Moda               8
Desvio_L1         21
Desvio_L2         22
Desvio_L3         65
Asimetria_L1      21
Asimetria_L2      22
Asimetria_L3      65
promGradiente      8
Relacion           8
Relacion_L1       21
Relacion_L2       21
Relacion_L3       22
Relacion_L4       65
EP               145
EPBinaria        145
RelEntropia  

**Limpieza de filas innecesarias**

Dentro de la base de datos hay filas que contienen el campo de principal interés (ACR) sin valores, es por ello que descartamos completamente estás filas.

In [162]:
_n = dataframe["ACR"].isna().sum()
print("Cantidad de filas sin campo ACR: ", _n, "porcentaje: {:.03%}".format(_n/dataframe.shape[0]))

Cantidad de filas sin campo ACR:  8 porcentaje: 0.121%


In [163]:
dataframe.dropna(axis=0,subset=["ACR"],inplace=True)

In [164]:
null_columns=dataframe.columns[dataframe.isnull().any()]
dataframe[null_columns].isnull().sum()

Dcm_18           14
Dcm_19           14
Desvio_L1        13
Desvio_L2        14
Desvio_L3        57
Asimetria_L1     13
Asimetria_L2     14
Asimetria_L3     57
Relacion_L1      13
Relacion_L2      13
Relacion_L3      14
Relacion_L4      57
EP              137
EPBinaria       137
RelEntropia     137
DF1              17
DF3              14
DF4              14
DF5              14
DF6              14
DF25              3
DFb1             17
DFb3             14
DFb4             14
DFb5             14
DFb6             14
DFb25             3
dtype: int64

Veamos términos relativos del impacto de datos faltantes en nuestro conjunto de datos: ?

In [166]:
len(dataframe.dropna())/len(dataframe)? 

SyntaxError: invalid syntax (<ipython-input-166-ac10fe2b4780>, line 1)

Lo que nos da que el impacto de los datos faltantes es de alrededor del 4%.  ? ver que onda con el formato de Mario

 VARIABLES CATEGORICAS
 

In [167]:
Features = [x for x in dataframe.columns if dataframe[x].dtype != 'float64']
print(Features)

['Imagen', 'View', 'Laterality', 'Dcm_5', 'Dcm_8', 'Dcm_15', 'Dcm_16', 'Dcm_23', 'ACR']


In [168]:
for feature in Features[1:]:
    #print(feature,np.unique(data[feature]),data[feature].dtype)
    print(feature,np.unique(dataframe[feature]))

View ['CC' 'MLO' 'MLOID']
Laterality ['L' 'R']
Dcm_5 ['ALUMINUM' 'MOLYBDENUM' 'RHODIUM' 'SILVER']
Dcm_8 ['MOLYBDENUM' 'TUNGSTEN']
Dcm_15 ['AUTOMATIC' 'AUTO_FILTER' 'MANUAL']
Dcm_16 ['AutoFilter' 'LORAD AUTO AEC' 'MANUAL' 'Manual']
Dcm_23 ['NO' 'YES']
ACR ['a' 'b' 'c' 'd']


Las variables 'View', 'Dcm_5', 'Dcm_15' y 'Dcm_16' poseen tres categorías. 'Laterality' y 'Dcm_23' poseen dos categorías mientras que 'ACR'y 'Dcm_16' tienen cuatro.


Siendo :

'Dcm_15'* (ExposureControlMode), modo de control de la exposición. 
    Existen tres posibles configuraciones: 
    * 'AUTOMATIC': Automático. 
    * 'AUTO_FILTER': Autofiltrado. 
    * 'MANUAL': Manual.
    
'Dcm_16'* (ExposureControlModeDescrip), descripción del modo de control de la exposición. 
    Existen tres posibles configuraciones: 
    * 'AutoFilter': Autofiltrado. 
    * 'MANUAL': Manual.
    * 'LORAD AUTO AEC' 
    * 'MANUAL' 
    * 'Manual'     


Notamos que "Manual" y "MANUAL" se corresponden al mismo atributo. 

In [123]:
dataframe["Dcm_16"].replace("Manual","MANUAL",inplace=True)


In [124]:
for feature in Features[1:]:
    #print(feature,np.unique(data[feature]),data[feature].dtype)
    print(feature,np.unique(dataframe[feature]))

View ['CC' 'MLO' 'MLOID']
Laterality ['L' 'R']
Dcm_5 ['ALUMINUM' 'MOLYBDENUM' 'RHODIUM' 'SILVER']
Dcm_8 ['MOLYBDENUM' 'TUNGSTEN']
Dcm_23 ['NO' 'YES']
ACR ['a' 'b' 'c' 'd']


De todas formas, por recomendación de experta, se descartan las columnas "Dcm_15" y "Dcm_16".

In [169]:
Features.remove('Dcm_15')
Features.remove('Dcm_16')

In [170]:
print(Features)

['Imagen', 'View', 'Laterality', 'Dcm_5', 'Dcm_8', 'Dcm_23', 'ACR']


Variables numéricas categóricas: son variables que, si bien son numéricas, presentan pocos valores entre su rango:

In [171]:
for feature in var_float:
    _u = np.unique(dataframe[feature])
    _n = len(_u)
    if _n < 4:
        print(feature,_n,_u)

Dcm_6 3 [0.03 0.05 0.7 ]
Dcm_7 1 [0.3]
Dcm_11 2 [660. 700.]
Dcm_13 2 [1.    1.073]
Dcm_21 2 [ 540. 2047.]
Dcm_22 2 [ 580. 4096.]


Como notamos en nuestra presentación anterior, las columnas correspondientes a las características
DICOM (aquí indicadas como 'Dicom_features'), no aportarían a la clasificación ACR 
las columnas: DCM_6,DCM_7, DCM_11, DCM_13, DCM_21 y DCM_22, algunas de éstas variables sólo se asignan a una de las categorías de la variable ACR, y en otros casos se asignan valores a dos de las 4 tipos de mamas, podría no aportar positivamente el hecho de provocar desbalances importantes al momento de predecir los tipos de mamas.

Siendo la descripción de las mismas:     

    * Dcm_6  --> FilterThickness. 
        Espesor del filtro
    *  Dcm_7  --> FocalSpots.
        Punto focal: Tamaño nominal del punto focal en mm utilizado para adquirir la imagen.
    *  Dcm_11  --> DistanceSourceDetector
        Distancia de la fuente al detector: Distancia en mm desde la fuente hasta el centro del detector
    *  Dcm_13  --> EstimatedMagnification : Estimación de la magnificación
    *  Dcm_21  --> WindowCenter
    *  Dcm_22  --> WindowWidth  

In [178]:
All_features = [x for x in dataframe.columns if dataframe[x].dtypes == 'float64']
Dicom_features = [x for x in dataframe.columns if ('Dcm_' in x and dataframe[x].dtypes == 'float64')]
#Dicom_features


In [179]:
# Las eliminamos del análisis de la base de datos:
Dicom_features.remove('Dcm_6')
Dicom_features.remove('Dcm_7')
Dicom_features.remove('Dcm_11')
Dicom_features.remove('Dcm_13')
Dicom_features.remove('Dcm_21')
Dicom_features.remove('Dcm_22')
#Se eliminan 6 columnas de DICOM

In [180]:
for columna, tipo in zip(dataframe.columns.values,dataframe.dtypes):
    print("%15s %9s" % (columna,tipo))

         Imagen    object
            Age   float64
           View    object
     Laterality    object
          Dcm_1   float64
          Dcm_2   float64
          Dcm_3   float64
          Dcm_4   float64
          Dcm_5    object
          Dcm_6   float64
          Dcm_7   float64
          Dcm_8    object
          Dcm_9   float64
         Dcm_10   float64
         Dcm_11   float64
         Dcm_12   float64
         Dcm_13   float64
         Dcm_14   float64
         Dcm_15    object
         Dcm_16    object
         Dcm_17   float64
         Dcm_18   float64
         Dcm_19   float64
         Dcm_20   float64
         Dcm_21   float64
         Dcm_22   float64
         Dcm_23    object
           Area   float64
       Promedio   float64
          Media   float64
         Desvio   float64
      Asimetria   float64
       Kurtosis   float64
           Moda   float64
      Desvio_L1   float64
      Desvio_L2   float64
      Desvio_L3   float64
   Asimetria_L1   float64
   Asimetria

### Rellenado de campos faltantes

Primero analizamos que porcentaje de campos faltantes hay.

In [146]:
null_columns=dataframe.columns[dataframe.isnull().any()]
dataframe[null_columns].isnull().sum()

Dcm_18           14
Dcm_19           14
Desvio_L1        13
Desvio_L2        14
Desvio_L3        57
Asimetria_L1     13
Asimetria_L2     14
Asimetria_L3     57
Relacion_L1      13
Relacion_L2      13
Relacion_L3      14
Relacion_L4      57
EP              137
EPBinaria       137
RelEntropia     137
DF1              17
DF3              14
DF4              14
DF5              14
DF6              14
DF25              3
DFb1             17
DFb3             14
DFb4             14
DFb5             14
DFb6             14
DFb25             3
dtype: int64

In [144]:
_ntot = dataframe.size
_nmissing = dataframe.isna().sum().sum()
print("Porcentaje de datos faltantes: {:.03%}".format(_nmissing/float(_ntot)))

Porcentaje de datos faltantes: 0.065%


### Por valores medios

In [145]:
data = dataframe.fillna(dataframe.mean())
print("Campos sin valores: ",data.isna().sum().sum())

Campos sin valores:  0


### Por vecinos cercanos

In [148]:
data_knn = dataframe.copy()

In [150]:
Mean = data_knn[var_float].mean()
Scale = data_knn[Nvar_float].std()

ValueError: Must pass DataFrame with boolean values only